In [1]:
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib

greens = matplotlib.cm.get_cmap('Greens')
reds = matplotlib.cm.get_cmap('Reds')
blues = matplotlib.cm.get_cmap('Blues')

from agrifoodpy.population.population_data import UN
from agrifoodpy.food.food_supply import FAOSTAT, scale_food, plot_food
from agrifoodpy.impact.impact import PN18
from agrifoodpy.impact import impact

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
FAOSTAT

<xarray.Dataset>
Dimensions:     (Region: 224, Year: 59, Item: 117)
Coordinates:
  * Region      (Region) int64 1 2 3 4 7 8 9 ... 5707 5801 5802 5803 5815 5817
  * Year        (Year) int64 1961 1962 1963 1964 1965 ... 2016 2017 2018 2019
  * Item        (Item) int64 2511 2513 2514 2515 2516 ... 2948 2949 2960 2961
Data variables: (12/16)
    kcal        (Region, Year, Item) float32 ...
    protein     (Region, Year, Item) float32 ...
    fat         (Region, Year, Item) float32 ...
    stock       (Region, Year, Item) float32 ...
    losses      (Region, Year, Item) float32 ...
    processing  (Region, Year, Item) float32 ...
    ...          ...
    domestic    (Region, Year, Item) float32 ...
    production  (Region, Year, Item) float32 ...
    feed        (Region, Year, Item) float32 ...
    seed        (Region, Year, Item) float32 ...
    imports     (Region, Year, Item) float32 ...
    exports     (Region, Year, Item) float32 ...

In [3]:
# Plant based energy intake scaling - 
# Animal origin products are scaled to keep energy instake constant

plant_scaling = 2

In [4]:
# area = 10 #Australia
area_pop = 826 #UK
area_fao = 229 #UK

# Select a year from the 1961-2019 range
year = 2019

In [5]:
population = UN.Medium.sel(Region=area_pop, Year=year)*1000

In [6]:
matching = pd.read_csv("PN18 and FAOSTAT list matching matrix.csv", sep=":")

alternative = matching["Alternative"]
alt_mask = ~np.isnan(alternative)

matching.drop("Alternative", axis=1, inplace=True)
matching.fillna(0, inplace=True)

# MAtching should only take one impact xarray variable as input
# issue a warning when more than one? 
# Take "alternative" as input too
# filling NaN should be made at the function level?
food_impact = impact.match(PN18, matching)
food_impact["GHG Emissions"].values[alt_mask] = alternative[alt_mask]
co2e_g = food_impact["GHG Emissions"]/1000

FileNotFoundError: [Errno 2] No such file or directory: 'PN18 and FAOSTAT list matching matrix.csv'

In [ ]:
elements_dict = {'production':0,
            'imports':1,
            'exports':2,
            'stock':3,
            'feed':4,
            'seed':5,
            'losses':6,
            'processing':7,
            'other':8,
            'food':9,}

food_items = food_impact.Item.values
food_uk = FAOSTAT.sel(Region=area_fao, Item=food_items, Year=year)

kcal_day = food_uk["kcal"]
prot_day = food_uk["protein"]
fats_day = food_uk["fat"]

food_uk[list(elements_dict.keys())] = food_uk[list(elements_dict.keys())]*1e9/population/365.25
food_uk = food_uk[list(elements_dict.keys())]

In [ ]:
Cereals = np.array([2511, 2513, 2514, 2515, 2516, 2517, 2518, 2520, 2531, 2532, 2533, 2534, 2535, 2807])
Pulses = np.array([2546, 2547, 2549, 2555])
Sugar = np.array([2536, 2537, 2541, 2542, 2543, 2558, 2562, 2570, 2571, 2572, 2573, 2574, 2576, 2577, 2578, 2579, 2580, 2581, 2582, 2586, 2745])
NutsSeed = np.array([2551, 2552, 2557, 2560, 2561])
VegetablesFruits = np.array([2563, 2601, 2602, 2605, 2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2625, 2641, 2775])
RuminantMeat =np.array([2731, 2732])
OtherMeat = np.array([2733, 2734, 2735, 2736])
Egg = np.array([2949])
Dairy = np.array([2740, 2743, 2948])
FishSeafood = np.array([2761, 2762, 2763, 2764, 2765, 2766, 2767, 2768, 2769])
Other = np.array([2630, 2633, 2635, 2640, 2642, 2645, 2655, 2656, 2657, 2658, 2680, 2737])
NonFood = np.array([2559, 2575, 2659, 2781, 2782])

groups = np.array([Cereals, Pulses, Sugar,
          NutsSeed, VegetablesFruits, RuminantMeat,
          OtherMeat, Egg, Dairy,
          FishSeafood, Other, NonFood], dtype=object)

In [ ]:
# Food groups are classified into plant and animal origin

plant_mask = np.arange(5)
animal_mask = np.arange(5, 10)
other_mask = np.arange(10, 12)

green_colors = greens(1/(2+plant_mask))
red_colors = reds(1/(2+animal_mask-5))
blue_colors = blues([0.5,1])

colors = np.concatenate((green_colors, red_colors, blue_colors))

In [ ]:
kcal_g = kcal_day / food_uk["food"]
prot_g = prot_day / food_uk["food"]
fats_g = fats_day / food_uk["food"]

kcal_day_food_uk = food_uk[elements_dict.keys()] * kcal_g
prot_day_food_uk = food_uk[elements_dict.keys()] * prot_g
fats_day_food_uk = food_uk[elements_dict.keys()] * fats_g
co2e_day_food_uk = food_uk[elements_dict.keys()] * co2e_g

kcal_day_food_uk = kcal_day_food_uk.where(np.isfinite(kcal_day_food_uk), other=0)
prot_day_food_uk = prot_day_food_uk.where(np.isfinite(prot_day_food_uk), other=0)
fats_day_food_uk = fats_day_food_uk.where(np.isfinite(fats_day_food_uk), other=0)
co2e_day_food_uk = co2e_day_food_uk.where(np.isfinite(co2e_day_food_uk), other=0)
# food_uk = food_uk.where(np.isfinite(food_uk), other=0)


In [ ]:
to_plot = [kcal_day_food_uk, prot_day_food_uk, fats_day_food_uk, food_uk, co2e_day_food_uk]
plot_food(to_plot, items=groups, colors = colors)

In [ ]:
origin = "imports"
fallback = "exports"
items = np.concatenate(groups[plant_mask])
food = kcal_day_food_uk
out_kcal_day_food_uk = scale_food(food=food, scale=plant_scaling, origin=origin, items=items, constant=True, fallback=fallback)

In [ ]:
scaling = out_kcal_day_food_uk/kcal_day_food_uk

out_food_uk = food_uk * scaling
out_prot_day_food_uk = prot_day_food_uk * scaling
out_fats_day_food_uk = fats_day_food_uk * scaling
out_co2e_day_food_uk = co2e_day_food_uk * scaling

In [ ]:
to_plot = [out_kcal_day_food_uk, out_prot_day_food_uk, out_fats_day_food_uk, out_food_uk, out_co2e_day_food_uk]
plot_food(to_plot, items=groups, colors=colors)
plt.show()

In [ ]:
plot_food([kcal_day_food_uk, out_kcal_day_food_uk], items=groups, colors=colors, sharex=True);